In [1]:
import os
import geopandas as gpd
import pandas as pd

In [11]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [5]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0"

In [6]:
enriched_list = os.listdir(enriched_path)

In [7]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [8]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [9]:
enriched_layers

{'point': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\CNRA_1950_2025.gdb',
   'layer_name': 'CNRA_enriched_20250512_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\NFPORS_1950_2025.gdb',
   'layer_name': 'NFPORS_enriched_20250509_point'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\Timber_Nonspatial_1950_2025.gdb',
   'layer_name': 'Timber_Nonspatial_20250523'}],
 'line': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\CalTRANS_1950_2025.gdb',
   'layer_name': 'CalTRANS_enriched_20250512'},
  {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\CNRA_1950_2025.gdb',
   'layer_name': 'CNRA_enriched_20250512_line'}],
 'polygon': [{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\BLM_1950_2025.gdb',
   'layer_name': 'BLM_enriched_20

In [13]:
timber_ns = gpd.read_file(r'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\V2.0\\Timber_Nonspatial_1950_2025.gdb',
                         driver = 'OpenFileGDB',
                         layer='Timber_Nonspatial_20250523')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [17]:
timber_ns[timber_ns.geometry == None].ACTIVITY_QUANTITY

4       0.0
9     500.0
17      5.0
Name: ACTIVITY_QUANTITY, dtype: float64

In [12]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-05-23 11:24:34,852 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-05-23 11:24:34,853 INFO  [process.append_polygon]  Concatenate all polygon records
2025-05-23 11:24:34,853 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250509' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_r

NameError: name 'exit' is not defined

In [9]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [10]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [11]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [12]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [13]:
timber_nonspatial_path

{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\Timber_Nonspatial_2020_2025.gdb',
 'layer_name': 'Timber_Nonspatial_20250221'}

In [14]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])
        

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [15]:
%%time



for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])
        
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-05-08 13:10:11,353 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-05-08 13:10:11,353 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-05-08 13:10:11,358 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape_Length' to 'Shape_Length_1'
  ogr_write(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape_Area' to 'Shape_Area_1'
  ogr_write(
2025-05-08 13:10:19,952 INFO  [pyogrio._io           ]  Created 62,088 records
2025-05-08 13:10:23,178 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-05-08 13:11:33,993 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-05-08 13:11:33,993 INFO  [utils.save_gdf_to_gdb ]        Chec

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field index_right of type Integer64 will be written as a Float64. To get Integer64, use layer creation option TARGET_ARCGIS_VERSION=ARCGIS_PRO_3_2_OR_LATER
  ogr_write(
2025-05-08 13:11:52,784 INFO  [pyogrio._io           ]  Created 103,837 records
2025-05-08 13:11:56,295 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
<timed exec>:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:12

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-05-08 13:11:58,935 INFO  [pyogrio._io           ]  Created 14,415 records
2025-05-08 13:11:59,055 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 16min 21s
Wall time: 2min 29s


In [16]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [24]:
enriched_points[enriched_points.AGENCY == 'TIMBER']

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,COUNTS_TO_MAS,index_right,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,Shape_Length,Shape_Area,geometry
14341,TI-0,TIMBER,None,None,None,None,TI-0,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-294794.243 -53503.055))
14342,TI-1,TIMBER,None,None,None,None,TI-1,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-294794.243 -53503.055))
14343,TI-2,TIMBER,None,None,None,None,TI-2,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-294794.243 -53503.055))
14344,TI-3,TIMBER,None,None,None,None,TI-3,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-284207.67 -54506.164))
14345,TI-4,TIMBER,None,None,None,None,TI-4,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-284207.67 -54506.164))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14410,TI-69,TIMBER,None,None,None,None,TI-69,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-293152.972 -48482.165))
14411,TI-70,TIMBER,None,None,None,None,TI-70,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-293152.972 -48482.165))
14412,TI-71,TIMBER,None,None,None,None,TI-71,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-284207.655 -50310.285))
14413,TI-72,TIMBER,None,None,None,None,TI-72,None,NaT,NaT,...,YES,NaN,None,None,None,NaN,NaN,NaN,NaN,MULTIPOINT ((-284207.655 -50310.285))


In [17]:
f_path =r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\temp_tables'

In [18]:
import pandas as pd

In [19]:
from datetime import datetime

In [20]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'US Department of Energy', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [21]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [22]:
save_gdf_to_gdb(activity_report_gdf, 
                r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb', 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-05-08 13:12:20,547 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-05-08 13:12:20,548 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-05-08 13:12:20,557 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-05-08 13:12:22,146 INFO  [pyogrio._io           ]  Created 121,225 records
2025-05-08 13:12:22,954 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250508
